# Installing Necessary Packages

In [2]:
!pip -q install zai-sdk
!pip -q install openai
!pip -q install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.3/123.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompa

# Import Packages

In [3]:
import os
import sys
import re
import time
import json
import random
import logging
import warnings
import requests

import numpy as np
import pandas as pd

from google import genai
from google.genai.types import GenerateContentConfig

from abc import ABC, abstractmethod

from zai import ZaiClient
from openai import OpenAI
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

# Logging

In [4]:
logger = logging.getLogger()      # root logger
logger.setLevel(logging.WARNING)

if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s %(levelname)s: %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# System Prompt

In [5]:
PROMPT_TEMPLATE = """
Your task is extracting medical entities from a single radiology report and returning them in a strictly defined JSON format.

========================
OUTPUT FORMAT (STRICT)
========================

Return a SINGLE JSON ARRAY. Each element corresponds to one input report and must follow this structure:

[
    {
        "general_finding": "None | [The broad finding category]",
        "specific_finding": "None | [Specific finding subtype related to general_finding]",
        "finding_presence": "None | present | absent | uncertain",
        "location": ["List of anatomical sites (strings)"],
        "degree": ["List of qualitative descriptors (strings)"],
        "measurement": "None | [numeric value (with or without unit) exactly as in text]",
        "comparison": "None | stable | improved | worsened"
    }
]

MANDATORY RULES:

- Output must be valid JSON.
- Output must be a SINGLE JSON array.

========================
ENTITY EXTRACTION RULES
========================

1. SOURCE TEXT
- Use only the report text provided.
- If the report has Findings and Impression sections, use ONLY those sections.
- If the report doesn't have the Findings or Impression sections use given text.
- Ignore History, Technique, Dose, Devices, administrative text, and boilerplate.

1. ENTITY COVERAGE (STRICT)
You MUST extract EVERY explicitly mentioned entity, including:
- Pathological findings
- Vascular findings (occlusion, thrombosis, patency, shunts)
- Postoperative or surgical status (e.g., surgically absent organs)
- Anatomic variants
- Explicit absence of findings (negated entities)
- Explicitly normal or unremarkable anatomy
- Explicit integrity or presence statements (e.g., intact, preserved)

Failure to extract explicitly stated entities is considered an error.

3. ENTITY DEFINITIONS

- general_finding:
  - A broad medical superclass (e.g., hemorrhage, fracture, mass, atrophy, atelectasis, effusion).
  - Must NOT include location, degree, or laterality unless it is an inherent part of the pathology name.
  - Use "None" ONLY for normal or non-pathological anatomy.

- specific_finding:
  - A more specific subtype of the general_finding (e.g. SDH, lacunar infarction, pleural effusion).
  - Must NOT be identical to general_finding.
  - Use "None" if no specific subtype is explicitly stated.

- finding_presence:
  - present → general_finding or specific_finding is explicitly stated as present.
  - absent → general_finding or specific_finding is explicitly searched for and not found (e.g., "no effusion", "no evidence of callus formation").
  - uncertain → equivocal language ("possible", "cannot exclude", "suspicious for").
  - None → ONLY for normal anatomy.

- location:
  - Anatomical region(s) where the general_finding, specific_finding or normal structure is described.

- degree:
  - Qualitative descriptors that encompass both the severity, extent, or normality of the general_finding/specific_finding and the state of health of the anatomical region/locations (e.g., mild, severe, unremarkable, intact, multilevel, patent).

- measurement:
  - Numeric values (size, volume, speed, etc.) exactly as written in the report.
  - Do NOT normalize or convert units. (e.g., "9 mm", "12.1 x 11.2 cm", "134 mm", "17 cm/sec", "1.5 cm", "%30")
  - Use "None" if no measurement is stated.

- "comparison":
  - Comparison with prior studies (stable, improved, worsened).
  - Use "None" if no comparison is stated.

1. SCHEMA CONSTRAINTS

- "general_finding" must be a superclass of "specific_finding".
- "specific_finding" must NOT be identical to "general_finding".
- "general_finding" and "specific_finding" entities cannot include "degree" and "anatomical laterality" or "location" unless it is an inherent part of the pathology name.
- The extracted entities must be in the report. Do not introduce new entities that doesn't  explicitly stated in the report.

5. NEGATIVE / ABSENT FINDINGS (MANDATORY)
If the report explicitly states absence, you MUST create an entity.

Examples:
- "No pleural effusion."
  → general_finding: "effusion"
    specific_finding: "pleural effusion"
    finding_presence: "absent"

- "Without evidence of fracture."
  → general_finding: "fracture"
    specific_finding: "None"
    finding_presence: "absent"

6. NORMAL / NON-PATHOLOGICAL FINDINGS (MANDATORY)
Explicitly normal anatomy MUST be extracted.

For normal findings:
- general_finding: "None"
- specific_finding: "None"
- finding_presence: "None"
- Include anatomy in location
- Include descriptors such as "normal", "unremarkable", "intact" in degree

Examples:
- "The spleen is normal in size."
- "Kidneys are unremarkable."
- "Bowel loops are intact."

{
    "general_finding": "None",
    "specific_finding": "None",
    "finding_presence": "None",
    "location": ["spleen"],
    "degree": ["normal in size"],
    "measurement": "None",
    "comparison": "None"
}

7. UNCERTAINTY HANDLING (MANDATORY)
If equivocal language is used, set finding_presence to "uncertain".

Uncertainty keywords include:
- possible
- cannot exclude
- suspicious for
- may represent
- favor

Example:
- "Cannot exclude capsular invasion."
  → finding_presence: "uncertain"

8.   ENTITY GRANULARITY
- One object per distinct entity.
- Do NOT merge unrelated findings.
- Separate affirmed and negated entities even if in the same sentence.

9.   SCHEMA CONSTRAINTS
- general_finding must be a superclass of specific_finding.
- general_finding and specific_finding must NOT contain degree or location terms.
- Do NOT introduce entities not explicitly stated in the report.

10.   FIELD VALUES
- Use literal "None" when no value applies.
- Use empty lists [] if no items exist.
- measurement must match text exactly.
- comparison must be "None" unless explicitly stated.

11.   DO NOT
- Do not infer laterality, severity, anatomy, or measurements.
- Do not guess diagnoses.
- Do not omit normal or absent findings.
- Do not hallucinate entities.
- Do not merge findings.

EXAMPLE 1
=========

Input Report:

5 non-rib-bearing lumbar vertebral bodies are present. Multilevel degenerative changes of the lumbar spine is noted with mild-to-moderate narrowing of the intervertebral disc spaces, most notable at L3-4, L4-5 and L5-S1. Vertebral body height loss is notable at L3. Small anterior osteophytes are noted at L 2, L3 and L4. Posterior facet arthropathy is moderate at L5-S1. The imaged lung bases are unremarkable. Large amount of stool is noted in the abdomen.

Extracted entities:

[
  {
    "general_finding": "None",
    "specific_finding": "None",
    "finding_presence": "None",
    "location": ["lumbar vertebral bodies"],
    "degree": ["non-rib-bearing", "5"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "degenerative change",
    "specific_finding": "None",
    "finding_presence": "present",
    "location": ["lumbar spine"],
    "degree": ["multilevel"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "narrowing",
    "specific_finding": "None",
    "finding_presence": "present",
    "location": ["intervertebral disc spaces", "L3-4", "L4-5", "L5-S1"],
    "degree": ["mild-to-moderate", "most notable"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "height loss",
    "specific_finding": "vertebral body height loss",
    "finding_presence": "present",
    "location": ["L3 vertebral body"],
    "degree": ["notable"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "osteophyte",
    "specific_finding": "anterior osteophytes",
    "finding_presence": "present",
    "location": ["L2", "L3", "L4"],
    "degree": ["small"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "arthropathy",
    "specific_finding": "posterior facet arthropathy",
    "finding_presence": "present",
    "location": ["L5-S1"],
    "degree": ["moderate"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "None",
    "specific_finding": "None",
    "finding_presence": "None",
    "location": ["lung bases"],
    "degree": ["unremarkable"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "stool",
    "specific_finding": "None",
    "finding_presence": "present",
    "location": ["abdomen"],
    "degree": ["large amount"],
    "measurement": "None",
    "comparison": "None"
  }
]

EXAMPLE 2
=========

Input Report:

Horizontal lucency through the mid L1 vertebral body is indicative of fracture. Similar lucency seen through the inferior C5 vertebral body, extending posteriorly to the articular facets. Multiple rib fractures are visualized, some of which appear to show large callus formation, indicating chronic fractures, others of which show no evidence of callus formation are more concerning for acute fractures. Specifically, these acute fractures are noted at the first ribs bilaterally as well as at the right second, fourth, and fifth ribs. Extensive degenerative change is visualized throughout the spine including a mild scoliotic curvature of the thoracolumbar spine.

Extracted entities:

[
    {
        "general_finding": "lucency",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "mid L1 vertebral body"
        ],
        "degree": [
            "horizontal"
        ],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "fracture",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "mid L1 vertebral body"
        ],
        "degree": [],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "lucency",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "inferior C5 vertebral body",
            "articular facets"
        ],
        "degree": [
            "extending posteriorly"
        ],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "fracture",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "inferior C5 vertebral body"
        ],
        "degree": [],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "fracture",
        "specific_finding": "rib fracture",
        "finding_presence": "present",
        "location": [
            "ribs"
        ],
        "degree": [
            "multiple"
        ],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "callus formation",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "ribs"
        ],
        "degree": [
            "large"
        ],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "fracture",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "ribs"
        ],
        "degree": ["chronic"],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "callus formation",
        "specific_finding": "None",
        "finding_presence": "absent",
        "location": [
            "ribs"
        ],
        "degree": [],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "fracture",
        "specific_finding": "acute fracture",
        "finding_presence": "uncertain",
        "location": [
            "ribs"
        ],
        "degree": [
            "concerning for"
        ],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "fracture",
        "specific_finding": "acute fracture",
        "finding_presence": "present",
        "location": [
            "first ribs bilaterally",
            "right second rib",
            "right fourth rib",
            "right fifth rib"
        ],
        "degree": [],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "degenerative change",
        "specific_finding": "None",
        "finding_presence": "present",
        "location": [
            "spine"
        ],
        "degree": [
            "extensive"
        ],
        "measurement": "None",
        "comparison": "None"
    },
    {
        "general_finding": "scoliosis",
        "specific_finding": "scoliotic curvature",
        "finding_presence": "present",
        "location": [
            "thoracolumbar spine"
        ],
        "degree": [
            "mild"
        ],
        "measurement": "None",
        "comparison": "None"
    }
]

====================
FINAL INSTRUCTION
====================

Extract medical entities from the given report:

{report}
"""

# API Keys

In [6]:
user_secrets = UserSecretsClient()

API_KEYS = {
    "gemini": user_secrets.get_secret("gemini_api_key_0"),
    "gemma": user_secrets.get_secret("gemini_api_key_0"),
    "glm": user_secrets.get_secret("glm_api_key"),
    "deepseek": user_secrets.get_secret("deepseek_api_key"),
}

# LLM Classes

In [7]:
class AIBaseModel(ABC):
    def __init__(self, api_key: str, model_name: str):
        self.api_key = api_key
        self.model_name = model_name
    
    @abstractmethod
    def invoke(self, prompt: str, **kwargs):
        raise NotImplementedError

In [8]:
class GeminiModel(AIBaseModel):
    def __init__(self, api_key: str, model_name: str = "gemini-2.5-flash"):
        self.model_name = model_name
        self.client = genai.Client(api_key=api_key)
        self.sleep_time = self._get_time_to_sleep()
        
    def _get_time_to_sleep(self):
        requests_per_minute = 15  # default
        
        if self.model_name == "gemini-2.5-flash":
            requests_per_minute = 5
        elif self.model_name == "gemini-3-flash-preview":
            requests_per_minute = 5
        elif self.model_name == "gemini-2.5-flash-lite":
            requests_per_minute = 10
        elif self.model_name == "gemini-1.5-flash":
            requests_per_minute = 15
        elif "gemma" in self.model_name:
            requests_per_minute = 30
            
        return 60 / requests_per_minute
    
    def invoke(
        self, 
        prompt: str,
        system_prompt: str | None = None,
        temperature: float = 0, 
        top_p: float = 1, 
        max_tokens: int = 8192,
    ):
        try:
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=prompt,
                config=GenerateContentConfig(
                    system_instruction=None,
                    temperature=temperature,
                    top_p=top_p,
                    max_output_tokens=max_tokens,
                ),
            )

            if hasattr(response, "candidates"):
                texts = []
                for c in response.candidates:
                    for p in getattr(c.content, "parts", []):
                        if getattr(p, "text", None):
                            texts.append(p.text)
                return "\n".join(texts) if texts else None
            return None
        except Exception as e:
            logger.error(f"Gemini API error: {e}")
        return None

In [9]:
class GLMModel(AIBaseModel):
    def __init__(self, api_key: str, model_name: str = "glm-4.5-flash"):
        super().__init__(api_key, model_name)
        self.client = ZaiClient(api_key=api_key)
        self.sleep_time = 6
        
    def invoke(
        self,
        prompt: str,
        system_prompt: str | None = None,
        temperature: float = 0.0,
        top_p: float = 1.0,
        max_tokens: int = 8192,
    ):
        try:
            messages = [
                {
                    "role": "system", 
                    "content": "You are a medical NLP system specialized in medical entity extraction from a given radiology report."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens,
                stream=False,
            )

            if response.choices:
                return response.choices[0].message.content.strip()

            return None
        except Exception as e:
            logger.error(f"GLM API error: {e}")
            if hasattr(e, "status_code"):
                logger.error(f"Status code: {e.status_code}")
            if hasattr(e, "body"):
                logger.error(f"Error body: {e.body}")
            return None

In [10]:
class DeepSeekModel(AIBaseModel):
    def __init__(self, api_key: str, model_name: str = "deepseek-chat"):
        super().__init__(api_key, model_name)
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.deepseek.com"
        )
        self.sleep_time = 3
        
    def invoke(
        self,
        prompt: str,
        system_prompt: str | None = None,
        temperature: float = 0.0,
        top_p: float = 1.0,
        max_tokens: int = 8192,
    ):
        try:
            messages = [
                {
                    "role": "system", 
                    "content": "You are a medical NLP system specialized in medical entity extraction from a given radiology report."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens,
                stream=False,
            )

            return response.choices[0].message.content.strip()
        except Exception as e:
            logger.error(f"DeepSeek API error: {e}")
            return None


## Get Model

In [11]:
def get_ai_model(model: str, model_name: str):
    model_map = {
        "gemini": GeminiModel,
        "glm": GLMModel,
        "deepseek": DeepSeekModel
    }
    
    if model not in model_map:
        raise ValueError(f"Unvalid model: {model}. Choices: {list(model_map.keys())}")
    
    return model_map[model](API_KEYS[model], model_name)

In [12]:
def load_jsonl(path: str):
    with open(path, encoding="utf-8") as file:
        return [json.loads(line) for line in file if line.strip()]

def load_json(path: str):
    with open(path, encoding="utf-8") as file:
        return json.load(file)

In [13]:
def build_prompt(report: str) -> str:
    return PROMPT_TEMPLATE.replace("{report}", report)

In [14]:
def safe_parse_json(text: str) -> dict | None:
    if not text:
        return None

    # 1. Markdown code fence temizle
    text = re.sub(r"```json|```", "", text, flags=re.IGNORECASE).strip()

    # 2. Direkt parse dene
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # 3. Fallback: ilk JSON objesini çek
    match = re.search(r"\{.*\}", text, re.S)
    if match:
        try:
            return json.loads(match.group())
        except Exception:
            return None

    return None


In [24]:
def run_inference_radgraph(
    dataset,
    output_path: str,
    model_id: str,
    model_name: str,
):
    model = get_ai_model(model_id, model_name)

    results = []
    for idx, sample in tqdm(enumerate(dataset), total=len(dataset), desc="Processing Samples"):
        
        prompt = build_prompt(sample["report"])

        raw_output = model.invoke(
            prompt=prompt,
            system_prompt=None
        )

        parsed = safe_parse_json(raw_output)

        print(f"Report: {sample['report']}")
        print(f"Parsed Output: {parsed}")
        
        result = {
            "dataset": sample["dataset"],
            "doc_key": sample["doc_key"],
            "report": sample["report"],
            "model": model_name,
            "entities": parsed,
        }
        
        results.append(result)

        if (idx + 1) % 5 == 0:
            temp_path = output_path.replace(".json", "_temp.json")
            with open(temp_path, "w") as f:
                f.write(json.dumps(results))
        

        time.sleep(model.sleep_time + 0.2)

    with open(output_path, "w") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    return results

def run_inference_ratener(
    dataset,
    output_path: str,
    model_id: str,
    model_name: str,
):
    model = get_ai_model(model_id, model_name)

    results = []
    for idx, sample in tqdm(enumerate(dataset), total=len(dataset), desc="Processing Samples"):
        
        prompt = build_prompt(sample["report"])

        raw_output = model.invoke(
            prompt=prompt,
            system_prompt=None
        )

        parsed = safe_parse_json(raw_output)

        print(f"Report: {sample['report']}")
        print(f"Raw Output: {raw_output}")
        
        result = {
            "note_id": sample["note_id"],
            "report": sample["report"],
            "model": model_name,
            "entities": parsed,
        }
        
        results.append(result)

        if (idx + 1) % 5 == 0:
            temp_path = output_path.replace(".json", "_temp.json")
            with open(temp_path, "w") as f:
                f.write(json.dumps(results))
        

        time.sleep(model.sleep_time + 0.2)

    with open(output_path, "w") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    return results

In [ ]:
INPUT_PATH = "/kaggle/input/rad-reports/ratener_mimic_train_reports.json"
OUTPUT_PATH = "/kaggle/working/ratener_mimic_train_mapped.json"

dataset = load_json(INPUT_PATH)[2:]

results = run_inference_ratener(
    dataset=dataset,
    output_path=OUTPUT_PATH,
    model_id="gemini",
    model_name="gemma-3-27b-it",
)

Processing Samples:   0%|          | 0/1136 [00:00<?, ?it/s]

Report: Three fluoroscopic views were obtained showing the insertion of a right-sided Port-A-Cath without the radiologist present. For more information, please refer to the procedure protocol.
Raw Output: ```json
[
  {
    "general_finding": "device",
    "specific_finding": "Port-A-Cath",
    "finding_presence": "present",
    "location": ["right side"],
    "degree": ["inserted"],
    "measurement": "None",
    "comparison": "None"
  }
]
```


Processing Samples:   0%|          | 1/1136 [00:10<3:12:05, 10.16s/it]

Report: The superior sagittal and transverse sinuses are patent and demonstrate normal flow signal. Slightly diminished flow signal in the right transverse sinus on MIP images but shows normal flow signal on the source images.. The deep venous system is also patent with normal flow signal in the internal cerebral veins, vein ___ and ___ sinus.
Raw Output: ```json
[
  {
    "general_finding": "None",
    "specific_finding": "None",
    "finding_presence": "None",
    "location": [
      "superior sagittal sinus",
      "transverse sinuses"
    ],
    "degree": [
      "patent"
    ],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "flow signal",
    "specific_finding": "None",
    "finding_presence": "None",
    "location": [
      "superior sagittal sinus",
      "transverse sinuses"
    ],
    "degree": [
      "normal"
    ],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "flow signal",
    "specific_finding": "

Processing Samples:   0%|          | 2/1136 [00:26<4:22:21, 13.88s/it]

...

Processing Samples:   4%|▍         | 47/1136 [18:10<9:38:48, 31.89s/it]

Report: The ventricles and extra-axial spaces are normal in size. There is no evidence of midline shift, mass effect, or hydrocephalus. Mild prominence of ventricles and sulci is seen including prominence of temporal horn indicating mild brain and medial temporal atrophy. There are mild changes of small vessel disease identified. Following gadolinium, no evidence of abnormal parenchymal, vascular or meningeal enhancement seen.
Raw Output: ```json
[
  {
    "general_finding": "None",
    "specific_finding": "None",
    "finding_presence": "None",
    "location": ["ventricles", "extra-axial spaces"],
    "degree": ["normal in size"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "shift",
    "specific_finding": "midline shift",
    "finding_presence": "absent",
    "location": [],
    "degree": ["no evidence of"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "mass effect",
    "specific_finding": "None",
    "fi

Processing Samples:   4%|▍         | 48/1136 [18:35<9:00:36, 29.81s/it]

Report: The intracranial vertebral and internal carotid arteries and their major branches appear normal without evidence of stenosis, occlusion, or aneurysm formation. Although the right posterior inferior cerebellar artery (___) is not identified, this may represent either occlusion or a vessel below the resolution of the study. A large area of acute infarction in the right cerebellar hemisphere is better imaged on this study from one day prior.
Raw Output: ```json
[
  {
    "general_finding": "artery",
    "specific_finding": "intracranial vertebral artery",
    "finding_presence": "None",
    "location": ["intracranial"],
    "degree": ["normal"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "artery",
    "specific_finding": "internal carotid artery",
    "finding_presence": "None",
    "location": ["intracranial"],
    "degree": ["normal"],
    "measurement": "None",
    "comparison": "None"
  },
  {
    "general_finding": "stenosis",
    "spe

Processing Samples:   4%|▍         | 49/1136 [18:54<8:03:16, 26.68s/it]

In [ ]:
# INPUT_PATH = "/kaggle/input/radgraph/stanford-radgraph-XL-sentence.jsonl"
# OUTPUT_PATH = "/kaggle/working/stanford-radgraph-XL-mapped.jsonl"

# dataset = load_jsonl(INPUT_PATH)[1:]

# results = run_inference_radgraph(
#     dataset=dataset,
#     output_path=OUTPUT_PATH,
#     model_id="deepseek",
#     model_name="deepseek-chat",
# )